# Inheritance: For Good or For Worse

- The pitfalls of subclassing from built-in types
- Multiple inheritance and the method resolution order

## Subclassing Built-In Types Is Tricky

-  code of the built-ins (written in C) does not call special methods overridden by user-defined classes.

In [3]:
class DoppleDict(dict):
    def __setitem__(self, key, value):
        return super().__setitem__(key, [value] * 2)

In [4]:
dd = DoppleDict(one=1)
dd

{'one': 1}

In [5]:
dd['two'] = 2 
dd

{'one': 1, 'two': [2, 2]}

In [7]:
dd.update(three=3)
dd

{'one': 1, 'two': [2, 2], 'three': 3}

1. The `__init__` method inherited from dict clearly ignored that `__setitem__` was overridden: the value of 'one' is not duplicated
2. The [] operator calls our `__setitem__` and works as expected: 'two' maps to the duplicated value [2, 2]
3. The update method from dict does not use our version of `__setitem__` either: the value of 'three' was not duplicated.

Subclassing built-in types like dict or list or str directly is errorprone because the built-in methods mostly ignore user-defined
overrides. Instead of subclassing the built-ins, `derive your classes from the collections module using UserDict, UserList, and UserString, which are designed to be easily extended`

In [8]:
import collections

class DoppleDict(collections.UserDict):
    def __setitem__(self, key, value):
        return super().__setitem__(key, [value] * 2)

In [10]:
dd = DoppleDict(one=1)
dd

{'one': [1, 1]}

In [11]:
dd['two'] = 2
dd

{'one': [1, 1], 'two': [2, 2]}

In [13]:
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

To summarize: the problem described in this section applies only to method delegation within the C language implementation of the built-in types.

## Multiple Inheritance and Method Resolution Order

Any language implementing multiple inheritance needs to deal with potential naming conflicts when unrelated ancestor classes implement a method by the same name

In [14]:
class A:
    def ping(self):
        print('ping', self)
      
        
class B(A):
    def pong(self):
        print('pong', self)
        

class C(A):
    def pong(self):
        print('PONG', self)
        

class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping:', self)
        
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

In [15]:
d = D()
d.pong()

pong <__main__.D object at 0x00000232FFA313D0>


In [16]:
C.pong(d)

PONG <__main__.D object at 0x00000232FFA313D0>


1. Simply calling d.pong() causes the B version to run.
2. You can always call a method on a superclass directly, passing the instance as an explicit argument.


The ambiguity of a call like d.pong() is resolved because Python follows a specific order when traversing the inheritance graph. That order is called MRO: Method Resolution Order. Classes have an attribute called `__mro__` holding a tuple of references to the superclasses in MRO order, from the current class all the way to the object class.

In [17]:
D.__mro__    

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [18]:
d.ping()

ping <__main__.D object at 0x00000232FFA313D0>
post-ping: <__main__.D object at 0x00000232FFA313D0>


1. The ping of D makes two calls.
2. The first call is super().ping(); the super delegates the ping call to class A; A.ping outputs this line.
3. The second call is print('post-ping:', self), which outputs this line.

In [19]:
d.pingpong()

ping <__main__.D object at 0x00000232FFA313D0>
post-ping: <__main__.D object at 0x00000232FFA313D0>
ping <__main__.D object at 0x00000232FFA313D0>
pong <__main__.D object at 0x00000232FFA313D0>
pong <__main__.D object at 0x00000232FFA313D0>
PONG <__main__.D object at 0x00000232FFA313D0>


1. Call #1 is self.ping(), which runs the ping method of D, which outputs this line and the next one.
2. Call #2 is super.ping(),which bypassesthe ping in D and findsthe ping method in A.
3. Call #3 is self.pong(), which finds the B implementation of pong, according to the `__mro__`.
4. Call #4 is super.pong(), which finds the same B.pong implementation, also following the `__mro__`.
5. Call #5 is C.pong(self), which finds the C.pong implementation, ignoring the `__mro__`.

The MRO takes into account not only the inheritance graph but also the `order in which superclasses are listed in a subclass declaration.`
- if the D class was declared as class D(C, B):, the `__mro__` of class D would be different: C would be searched before B.

## Coping with Multiple Inheritance

### 1. Distinguish Interface Inheritance from Implementation Inheritance

-  Inheritance of interface creates a subtype, implying an “is-a relationship
-  Inheritance of implementation avoids code duplication by reuse.

`Inheritance for code reuse is an implementation detail, and it can often be replaced by composition and delegation`. On the other hand, interface inheritance is the backbone of a framework.

### 2. Make Interfaces Explicit with ABCs

- In Python ≥ 3.4, this means: subclass abc.ABC

### 3. Use Mixins for Code Reuse

If a class is designed to `provide method implementations for reuse by multiple unrelated subclasses`, without implying an “is-a” relationship, it should be an explicit `mixin` class.


- A mixin should never be instantiated, and concrete classes should not inherit only from a mixin. 
- Each mixin should provide a single specific behavior, implementing few and very closely related methods

### 4. Make Mixins Explicit by Naming

There is no formal way in Python to state that a class is a mixin, so it is highly recommended that they are named with a `…Mixin suffix`

### 5. An ABC May Also Be a Mixin; The Reverse Is Not True

- Because an ABC can implement concrete methods, it works as a mixin as well. 

### 6. Don’t Subclass from More Than One Concrete Class

- Concrete classes should have zero or at most one concrete superclass
- all but one of the superclasses of a concrete class should be ABCs or mixins.

### 7. Provide Aggregate Classes to Users

- If some combination of ABCs or mixins is particularly useful to client code, provide a class that brings them together in a sensible way

### 8. “Favor Object Composition Over Class Inheritance.”

- favoring composition leads to more flexible designs
- Composition and delegation can replace the use of mixins to make behaviors available to different classes, but cannot replace the use of interface inheritance to define a hierarchy of types.

# Chapter Summary

- explained issues with subclassing built=in types:  their native methods implemented in C do not call overridden methods in subclasses
- t’s easier to subclass UserList, UserDict, or UserString—all defined in the collections module
- saw how the method resolution order, encoded in the `__mro__` class attribute, addresses the problem of potential naming conflicts in inherited methods
- Also explore how to cope with multiple inheritances. And why composition is favored over inheritance